In [5]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import pandas as pd
import simpy
import numpy as np
import random
from datetime import datetime

app = Flask(__name__)
CORS(app)

# Load dataset once
df = pd.read_csv('C:\\Users\\shash\\Downloads\\airport.csv')
df = df.dropna(subset=['Passenger_Count', 'ReportPeriod'])

# Convert dates with error handling
def safe_date_parse(x):
    try:
        return pd.to_datetime(x)
    except:
        return pd.NaT

df['ReportPeriod'] = df['ReportPeriod'].apply(safe_date_parse)
df = df.dropna(subset=['ReportPeriod'])

@app.route('/simulate', methods=['POST'])
def simulate():
    try:
        data = request.json
        terminal = data.get('terminal', 'Terminal B')
        num_counters = int(data.get('num_counters', 5))
        sim_duration = int(data.get('sim_duration', 960))

        # Filter Terminal
        filtered_df = df[df['Terminal'] == terminal]
        if filtered_df.empty:
            fallback_terminal = df['Terminal'].mode()[0]
            filtered_df = df[df['Terminal'] == fallback_terminal]

        # Monthly aggregation with error handling
        monthly_data = (filtered_df.groupby(pd.Grouper(key='ReportPeriod', freq='M'))
                       ['Passenger_Count'].sum().reset_index())
        monthly_data['Daily_Passengers'] = monthly_data['Passenger_Count'] / 30
        monthly_data = monthly_data.dropna(subset=['Daily_Passengers'])
        
        if monthly_data.empty:
            return jsonify({"error": "No valid monthly data available"}), 400

        avg_daily_passengers = int(monthly_data['Daily_Passengers'].mean())

        # Simulation setup
        arrival_rate = avg_daily_passengers / (16 * 60)
        service_time = 3
        service_rate = 1 / service_time

        env = simpy.Environment()
        counters = [simpy.Resource(env, capacity=1) for _ in range(num_counters)]

        # Tracking variables
        queue_lengths = [[] for _ in range(num_counters)]
        service_times = []
        wait_times = []
        counter_utilization = [0] * num_counters

        def passenger(env, name, counters):
            arrival_time = env.now
            queue_sizes = [len(counter.queue) for counter in counters]
            chosen_idx = np.argmin(queue_sizes)
            queue_lengths[chosen_idx].append(queue_sizes[chosen_idx])

            with counters[chosen_idx].request() as req:
                yield req
                wait_time = env.now - arrival_time
                wait_times.append(wait_time)

                service_duration = random.expovariate(service_rate)
                service_times.append(service_duration)
                counter_utilization[chosen_idx] += service_duration
                yield env.timeout(service_duration)

        def generator(env, arrival_rate, counters):
            i = 0
            while True:
                yield env.timeout(random.expovariate(arrival_rate))
                env.process(passenger(env, f'Passenger {i}', counters))
                i += 1

        env.process(generator(env, arrival_rate, counters))
        env.run(until=sim_duration)

        results = {
            "max_queue_lengths": [max(q) if q else 0 for q in queue_lengths],
            "average_service_time": round(np.mean(service_times), 2),
            "average_wait_time": round(np.mean(wait_times), 2),
            "counter_utilization": [(u / sim_duration) * 100 for u in counter_utilization]
        }

        return jsonify(results)

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)  # Disable reloader for Jupyter compatibility


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/Apr/2025 01:03:29] "OPTIONS /simulate HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2025 01:03:30] "POST /simulate HTTP/1.1" 200 -


In [3]:
%tb

SystemExit: 1